#**STEP 7**

In [1]:
import os

current_path = os.getcwd()
print("Current Path:", current_path)

Current Path: C:\Users\panos


In [2]:
# Adding the neccecary imports to our code
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydot # conda install pydot
import time
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
import math

In [3]:
#type of data:
url_Les_Miserables_network  = 'Downloads/LesMiserables.dot'
url_Rome_graph  = '../data/rome.dot'
url_Jazz_Network  = 'Downloads/JazzNetwork.dot'
url_Small_Directed_Network  = 'Downloads/noname.dot'
url_Pro_League_Network  = 'Downloads/LeagueNetwork.dot'
url_Argumentation_network  = '../data/devonshiredebate_withclusters.dot'
url_Political_blogosphere_network  = '../data/polblogs.dot'
#put here the url you want
url = url_Les_Miserables_network

In [4]:
# classes
class Vertex:
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y
        self.neighbors = []

        self.parents = []
        self.children = []

    def add_neighbor(self,neighbor):
        self.neighbors.append(neighbor)
    def add_parents(self,parent):
        self.parents.append(parent)
    def add_children(self,child):
        self.children.append(child)


class Edge:
    def __init__(self, start, end, weight=0):
        self.start = start
        self.end = end
        #############################
        self.weight = weight
        ###########################

class Graph:
    def __init__(self):
        self.vertices = []
        self.edges = []

    def add_vertex(self, vertex):
        self.vertices.append(vertex)

    def add_edge(self, edge):
        self.edges.append(edge)

    def get_vertex(self, id):
        for vertex in self.vertices:
            if vertex.id == id:
                return vertex

    def remove_edge(self, start, end):
        for edge in self.edges:
            if edge.start == start and edge.end == end:
                self.edges.remove(edge)

    def remove_vertex(self, id):
        for vertex in self.vertices:
            if vertex.id == id:
                self.vertices.remove(vertex)

    def update_x_y(self,id,x,y):
      for vertex in self.vertices:
            if vertex.id == id:
              vertex.x = x
              vertex.y = y


    def visualize_graph(self, custom_figsize, radius):  # O(V + V + E) = O(2V + E) = O(V + E)

      self.place_vertices_in_circle(radius)  # O(V)

      fig, ax = plt.subplots(figsize=(custom_figsize, custom_figsize))  # Adjust width and height as needed
      # Plot Vertices
      scaling_factor = 1 / len(self.vertices)
      for vertex in self.vertices:  # O(V)
          ax.scatter(vertex.x, vertex.y, color='blue', zorder=2, s=scaling_factor * 40000 * (custom_figsize / 15),
                    edgecolors='black')
          ax.text(vertex.x, vertex.y, str(vertex.id), fontsize=(scaling_factor * 40000 * (custom_figsize / 15)) / 100,
                  ha='center', va='center', zorder=len(self.vertices) * scaling_factor * 2, color='white')
      # Plot edges with direction
      for edge in self.edges:  # O(E)
          start = edge.start
          end = edge.end
          dx = end.x - start.x
          dy = end.y - start.y
          ax.annotate("", xy=(end.x, end.y), xytext=(start.x, start.y),
                      arrowprops=dict(arrowstyle="->", linewidth=2.0, color='gray'))

      ax.set_xlabel('X')
      ax.set_ylabel('Y')
      ax.set_title('Graph Visualization with Edge Direction')
      ax.grid(True)
      #plt.show()
     # plt.close(fig)


    def place_vertices_in_circle(self,radius): # O(V)
        num_vertices = len(self.vertices)
        radius = radius  # Radius of the circle
        center_x = 0
        center_y = 0
        angle_increment = 2 * np.pi / num_vertices

        # Place vertices in a bit of randomized circle
        for i, vertex in enumerate(self.vertices): # O(V)
            angle = i * angle_increment
            vertex.x = center_x + radius * np.cos(angle)
            vertex.y = center_y + radius * np.sin(angle)




In [5]:

def create_custom_graph(G): # O(V) + O(E) + O(V) + O(E) = O(2V + 2E) = O(V + E)
    # Parsing the vertices and edges from the generated graph into lists
    vertices = [n.get_name() for n in G.get_node_list()] # O(V)

    edges = [(edge.get_source(),edge.get_destination(),edge.get_attributes()) for edge in G.get_edge_list()] # O(E)
    graph = Graph()
    for vertex in vertices: # O(V)
      vertex = Vertex(vertex, 0, 0)  # Initialize the vertex with ID and default coordinates
      graph.add_vertex(vertex)

    for edge in edges: # O(E)
        w=0
        if ("JazzNetwork" not in url):
             w = list(edge[2].values())[0].strip("\"")
        if (float(w)<1.0 or float(w)>3.0) and ("LeagueNetwork" in url): # because only prolegue needs fltering
            continue
        source_vertex = graph.get_vertex(edge[0])
        neighbor_vertex = graph.get_vertex(edge[1])

        #print("attributes",list(edge[2].values())[0].strip("\""))

        if source_vertex is None:
            source_vertex = Vertex(edge[0], 0, 0)
        if neighbor_vertex is None:
            neighbor_vertex = Vertex(edge[1], 0, 0)
        source_vertex.add_neighbor(neighbor_vertex)
        source_vertex.add_children(neighbor_vertex)
        neighbor_vertex.add_parents(source_vertex)
        ##############################################################
        graph.add_edge(Edge(source_vertex, neighbor_vertex,float(w)))

        ##############################################################
    if ("JazzNetwork" in url) : # as the jazz network dont have weight initialise jazznetwork edges weight with 1
        for edge in graph.edges:
            edge.weight = 1.0
    return graph

Step 3

In [6]:
class Forced_vertex:
    def __init__(self, id):
        self.id = id
        self.position = np.array([np.random.uniform(0, 10), np.random.uniform(0, 10)])  # Random initial position
        self.x = self.position[0]####### new
        self.y = self.position[1]########## new
        self.force = np.zeros(2)  # Initial force vector
def update_vertices_in_OG(Forced_vertex,graph):
    for v in graph.vertices:
       if Forced_vertex.id == v.id:
          v.x=Forced_vertex.position[0]
          v.y=Forced_vertex.position[1]

def initialize_Forced_vertices(graph):
    vertices = {v.id: Forced_vertex(v.id) for v in graph.vertices}

    return vertices

def compute_forces(vertices, edge,edge_set, Cspring=2.0, Crep=1.0, l=1.0):
    for vertex in vertices.values():
        vertex.force.fill(0)

    for vertex_source in vertices.values():
        for vertex_target in vertices.values():
            if vertex_source.id != vertex_target.id:
                if not is_edge_between_vertices(vertex_source.id, vertex_target.id, edge_set): # Calculate repulsive forces between all nodes
                    direction = vertex_target.position - vertex_source.position
                    distance = np.linalg.norm(direction)
                    if distance > 0:
                        unit_vector = direction / distance
                        force = (Crep / distance ** 2) * unit_vector
                        vertex_target.force += force
                else: # Calculate atractive forces between all nodes
                    direction = vertex_source.position - vertex_target.position
                    distance = np.linalg.norm(direction)
                    if distance > 0:
                        unit_vector = direction / distance
                        force = Cspring * np.log10(distance / l) * unit_vector
                        vertex_target.force += force
def is_edge_between_vertices(source_id, target_id, edge_set):
    return (source_id, target_id) in edge_set or (target_id, source_id) in edge_set

def update_positions(vertices, dt=0.1, damping=0.99):
    max_movement = 0.0
    for vertex in vertices.values():
        movement = np.linalg.norm(vertex.force * dt)
        vertex.position += vertex.force * dt
        vertex.x=vertex.position[0]
        vertex.y=vertex.position[1]
        vertex.force *= damping
        max_movement = max(max_movement, movement)
    return max_movement

def visualize_layout(ax,vertices, edges,title):
    # plt.figure(figsize=(20, 20))
    norms = [np.linalg.norm(vertex.force) for vertex in vertices.values()]
    norm = Normalize(vmin=min(norms), vmax=max(norms))
    for edge in edges:
        source_pos = vertices[edge.start.id].position
        target_pos = vertices[edge.end.id].position
        ax.plot([source_pos[0], target_pos[0]], [source_pos[1], target_pos[1]], 'k-', linewidth=0.2)
    for vertex in vertices.values():
        color_rgba = plt.cm.YlOrRd(norm(np.linalg.norm(vertex.force)))
        color_hex = mcolors.rgb2hex(color_rgba[:3])
        ax.plot(vertex.position[0], vertex.position[1], 'o', color=color_hex)
        ax.text(vertex.position[0], vertex.position[1], str(vertex.id), fontsize=8, ha='center', va='bottom', color="green")
        ax.set_title(title)
def create_forced_graph(ax,graph,num_iterations=1000,Cspring = 2.0, Crep=1.0, l=1.0,e=0.05,learning_rate=0.1,title = "Force directed Les Miserables network"):
    vertices = initialize_Forced_vertices(graph)
    edges = graph.edges
    edge_set = {(edge.start.id, edge.end.id) for edge in edges}
    k = 0
    max_movement = 1
    while k < num_iterations and max_movement > e:
        compute_forces(vertices, edges,edge_set)
        max_movement = update_positions(vertices,learning_rate)
        for v in vertices.values():######## new
          update_vertices_in_OG(v,graph)######## new
        k += 1
        if max_movement <= e:
            print(k)
    visualize_layout(ax,vertices, edges,title)


Step 4

In [ ]:
# Parsing the data from the dot file:
G = pydot.graph_from_dot_file(url)[0]
#Convering the graph into our custom graph
graph2 = create_custom_graph(G) #O(V) + O(E) = O(V + E)
# Visualize the graph

fig_size=5+len(graph2.vertices)/6 # new line
radius=5+len(graph2.vertices)/4 # new line

graph2.visualize_graph(fig_size,radius) #O(V + E)

In [ ]:
import sys
sys.setrecursionlimit(10000)
def remove_sink_vertices(G, h_graph):
  sink = 1
  while sink == 1:

      sink = 0
      for v1 in G.vertices:

        #if it doesnt have children then is sink add them to the sink graph
        if not v1.children:
          sink = 1

          #add sink node to the new graph
          h_graph.add_vertex(v1)

          #remove node from original
          G.remove_vertex(v1.id)

          for v2 in v1.parents:
            #Add sink edges to the new graph
            h_graph.add_edge(Edge(v2, v1 ))

            #remove edges from original
            G.remove_edge(v2, v1)
            #remove v1 as parent of v2
            v2.children.remove(v1)
  return G, h_graph



def remove_source_vertices(G, h_graph):
  source = 1
  while source == 1:
      source = 0
      for v1 in G.vertices:

        #if it doesnt have children then is sink add them to the sink graph
        if not v1.parents:
          source = 1

          #add sorce node to the new graph
          h_graph.add_vertex(v1)
          #remove node from original
          G.remove_vertex(v1.id)

          for v2 in v1.children:

            #Add source edge to the new graph
            h_graph.add_edge(Edge(v1, v2))

            #remove edges from original
            G.remove_edge(v1, v2)
            #remove v1 as parent of v2
            v2.parents.remove(v1)

  return G, h_graph

def remove_maximal_vertex(G, h_graph, reverse_edges, x):

  if bool(G.vertices):

      #Choose vertex with maximum
      current_vertex = G.vertices[0]
      max = abs(len(current_vertex.parents)-len(current_vertex.children))
      for v1 in G.vertices:
        cur = abs(len(v1.parents)-len(v1.children))

        # print(v1.id, " ", max)
        # if x == 1:
        #   if v1.id == "17":
        #     max = cur
        #     current_vertex = v1
        #     break
        if cur > max:
          max = cur
          current_vertex = v1
        if cur == max:
          if len(v1.parents) < len(current_vertex.parents):
            max = cur
            current_vertex = v1

      #add current vertex to the new graph
      h_graph.add_vertex(current_vertex)

      #remove current vertex from original
      G.remove_vertex(current_vertex.id)

      for c in current_vertex.children:
        #Add source edge to the new graph
        h_graph.add_edge(Edge(current_vertex, c))

        #remove edges from original
        G.remove_edge(current_vertex, c)
        #remove v1 as parent of v2
        c.parents.remove(current_vertex)

      for p in current_vertex.parents:
        reverse_edges.append(Edge(p, current_vertex))

  return G, h_graph, reverse_edges

def heuristic_guarantees(G):

  h_graph = Graph()
  reverse_edges = []
  x=1
  while G.vertices:

    G, h_graph = remove_sink_vertices(G, h_graph)

    G, h_graph = remove_source_vertices(G, h_graph)

    G, h_graph, reverse_edges = remove_maximal_vertex(G,h_graph, reverse_edges, x)
    x=0

  #add the reversed edges
  for reverse in reverse_edges:
    h_graph.remove_edge(reverse.start, reverse.end)
    h_graph.add_edge(Edge(reverse.end, reverse.start))

  return(h_graph,reverse_edges)

import copy
copied_graph = copy.deepcopy(graph2)
dag, reverse_edges = heuristic_guarantees(copied_graph)

# Visualize the graph
fig_size=5+len(dag.vertices)/6 # new line
radius=5+len(dag.vertices)/4 # new line
dag.visualize_graph(fig_size,radius) #O(V + E)

################ for some reasone the cyclic visualization doesnt work even though the graph is correct
for v in dag.vertices:
  if v.id == '"\\n"':
      dag.vertices.remove(v)

In [9]:

def update_parents_children(G):

  for vertex in G.vertices:
    vertex.parents.clear()
    vertex.children.clear()

  for edge in G.edges:
    edge.start.add_children(edge.end)
    edge.end.add_parents(edge.start)

  return(G)

def sources_of_graph(G):
  sources = []
  for v in G.vertices:
    if not v.parents:
      sources.append(v)

  return(sources)

def update_graph(G, sources):

  for v in sources:
    G.remove_vertex(v.id)
    for c in v.children:
      #remove edges from original
      G.remove_edge(v, c)
      #remove v as parent of c
      c.parents.remove(v)
  return(G)

def layer_assignment(G):

  sources = sources_of_graph(G)
  i=1
  layers = []
  i =0
  while sources:
    layers.append(sources)
    G = update_graph(G, sources)
    sources = sources_of_graph(G)

  return(layers)

copied_graph = copy.deepcopy(dag)
copied_graph = update_parents_children(copied_graph)

layers = layer_assignment(copied_graph)

In [ ]:

def visualize_layerd_graph(layers, G, fig_size, radius):
  fig, ax = plt.subplots(figsize=(fig_size, fig_size))  # Adjust width and height as needed
  # Plot Vertices
  scaling_factor = 1 / len(G.vertices)
  for idx,layer in enumerate(layers):
      for idx1,vertex in enumerate(layer):
          G.update_x_y(vertex.id, idx1, idx)
          ax.scatter(idx1, idx, color='blue', zorder=2, s=scaling_factor * 40000 * (fig_size / 15),
                    edgecolors='black')
          ax.text(idx1, idx, str(vertex.id), fontsize=(scaling_factor * 40000 * (fig_size / 15)) / 100,
                  ha='center', va='center', zorder=len(G.vertices) * scaling_factor * 2, color='white')

  for edge in G.edges:
        start = edge.start
        end = edge.end
        dx = end.x - start.x
        dy = end.y - start.y
        ax.annotate("", xy=(end.x, end.y), xytext=(start.x, start.y),
                    arrowprops=dict(arrowstyle="->", linewidth=1.5, color='gray'))

  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_title('Graph Visualization with Edge Direction')
  ax.grid(True)
  plt.show()


copied_graph = copy.deepcopy(dag)
fig_size=5+len(copied_graph.vertices)/6 # new line
radius=5+len(copied_graph.vertices)/4 # new line

visualize_layerd_graph(layers, copied_graph, fig_size, radius)



In [11]:
def insert_dummies(layers, G):

  for idx, layer in enumerate(layers):
    for idx1, vertex in enumerate(layer):
      G.update_x_y(vertex.id, idx1, idx)

  dummy_num = 0
  #Add dummy node and esdges
  edges = G.edges[:]
  for edge in edges:
      start = edge.start
      end = edge.end


      if abs(start.y - end.y) > 1:  # Check if the edge spans more than 1 layer
          # Choose the layer for the dummy node
          numberOfDum=abs(start.y - end.y)-1
          add_factor= 0

          prev_vertex = start
          next_vertex = None

          while numberOfDum > 0:

              add_factor =add_factor + 1
              dummy_layer = min(start.y, end.y) + add_factor  # Place the dummy node in the middle of the layers

              #create the dummy
              dummy_id ="D"+ str(dummy_num)+"_s"+str(start.id)+"_e"+str(end.id)
              dummy_vertex = Vertex(dummy_id, 0, 0)
              G.add_vertex(dummy_vertex)


              #put the dummy in the right layer over its parent
              layers[dummy_layer].insert(start.x, dummy_vertex)


              #add the edges
              next_vertex = dummy_vertex
              G.add_edge(Edge(prev_vertex,next_vertex))

              prev_vertex = next_vertex

              numberOfDum = numberOfDum - 1
              dummy_num+=1

          G.add_edge(Edge(prev_vertex,end))
          #remove the direct node
          G.remove_edge(start, end)

  return layers, G

layered_dummies_graph = copy.deepcopy(dag)
layers, layered_dummies_graph = insert_dummies(layers, layered_dummies_graph)
print("count nodes",len(layered_dummies_graph.vertices),"count edges",len(layered_dummies_graph.edges))


count nodes 867 count edges 1044


In [ ]:

def visualize_layered_graph_with_dummies(layers, G, fig_size, radius):

    fig, ax = plt.subplots(figsize=(fig_size, fig_size))  # Adjust width and height as needed

    # Plot Vertices
    scaling_factor = 1 / len(G.vertices)
    for idx,layer in enumerate(layers):
        for idx1,vertex in enumerate(layer):
            #update_x_y
            G.update_x_y(vertex.id, idx1, idx)
            if vertex.id.startswith("D"):
              ax.scatter(idx1, idx, color='red', zorder=2, s=(scaling_factor * 40000 * (fig_size / 15))/1.2,
                        edgecolors='black')
              ax.text(idx1, idx, "D", fontsize=(scaling_factor * 40000 * (fig_size / 15)) / 100,
                      ha='center', va='center', zorder=len(G.vertices) * scaling_factor * 2, color='black')
            else:
              ax.scatter(idx1, idx, color='blue', zorder=2, s=scaling_factor * 40000 * (fig_size / 15),
                      edgecolors='black')
              ax.text(idx1, idx, str(vertex.id), fontsize=(scaling_factor * 40000 * (fig_size / 15)) / 50,
                    ha='center', va='center', zorder=len(G.vertices) * scaling_factor * 2, color='white')

    for edge in G.edges:
        start = edge.start
        end = edge.end
        dx = end.x - start.x
        dy = end.y - start.y
        ax.annotate("", xy=(end.x, end.y), xytext=(start.x, start.y),
                    arrowprops=dict(arrowstyle="->", linewidth=1.5, color='gray'))

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('layered graph for',)
    ax.grid(True)
    plt.show()




fig_size=5+len(layered_dummies_graph.vertices)/6 # new line
radius=5+len(layered_dummies_graph.vertices)/4 # new line


visualize_layered_graph_with_dummies(layers, layered_dummies_graph, fig_size, radius)




In [13]:
#GENERAL FUNTIONS######################################################
from itertools import permutations

def update_parents_children_in_layers(layers, edges):
  for layer in layers:
    for vertex in layer:
      vertex.parents.clear()
      vertex.children.clear()

  for layer in layers:
    for vertex in layer:
      for edge in edges:
        if vertex.id == edge.end.id:
          vertex.parents.append(edge.start)

        if vertex.id == edge.start.id:
          vertex.children.append(edge.end)

def get_index(layer, target_id):
    for idx, vertex in enumerate(layer):
        if vertex.id == target_id:
            return idx
    return None

def calculate_total_crossing(layers):

    crossing_count = 0
    #for each pair of consecutive layers
    for idx, layer in enumerate(layers[:-1]):
        first_layer = layer
        second_layer = layers[idx+1]

        for i in range(len(second_layer)):
          for j in range(i + 1, len(second_layer)):

            for parent_v1 in second_layer[i].parents:
              #get the index of the parent of the first node
              pv1_idx = get_index(first_layer, parent_v1.id)

              for parent_v2 in second_layer[j].parents:
                #get the index of the parent of the second node
                pv2_idx = get_index(first_layer, parent_v2.id)

                #if the parent of v1 is after the parent of v2 it means there is a crossing
                if pv1_idx > pv2_idx:
                  crossing_count +=1
    return(crossing_count)


In [14]:
#BARYCENTER #################################################
def rearange_of_layers_Barycenter(first_layer, second_layer):

  for vertex in second_layer:

    deg = len(vertex.parents)
    sum = 0
    for parent in vertex.parents:
      sum += get_index(first_layer, parent.id)

    if deg == 0:
      continue
    vertex_x = 1/deg * sum

    vertex.x = vertex_x

  second_layer = sorted(second_layer, key=lambda vertex: vertex.x)
  return (second_layer)


def rearange_of_layers_reverse_Barycenter(first_layer, second_layer ):

  for vertex in first_layer:
    deg = len(vertex.children)
    sum = 0
    for child in vertex.children:
      sum += get_index(second_layer, child.id)

    if deg == 0:
      continue
    vertex_x = 1/deg * sum

    vertex.x = vertex_x

  first_layer = sorted(first_layer, key=lambda vertex: vertex.x)
  return (first_layer)


In [15]:
#MEDIAN ###################################################################
def custom_sort_key_a(vertex):

  if (len(vertex.parents)) % 2 == 1 :
      return (vertex.x, 0)
  else:
      return (vertex.x, 1)

def custom_sort_key_b(vertex):
    if (len(vertex.children) ) % 2 == 1 :
        return (vertex.x, 0)
    else:
        return (vertex.x, 1)

def rearange_of_layers_median(first_layer, second_layer):

  for vertex in second_layer:

    parents_idx = []
    for parent in vertex.parents:
      parents_idx.append(get_index(first_layer, parent.id))

    parents_idx = sorted(parents_idx)

    if len(parents_idx) == 0:
      new_position = 0
    else:
      new_position = parents_idx[math.ceil((len(parents_idx)) / 2) - 1]

    vertex.x = new_position

  second_layer = sorted(second_layer, key=custom_sort_key_a)

  return (second_layer)

def rearange_of_layers_median_reverse(first_layer, second_layer):

  for vertex in first_layer:

    children_idx = []
    for child in vertex.children:
      children_idx.append(get_index(second_layer, child.id))

    children_idx = sorted(children_idx)

    if len(children_idx) == 0:
      new_position = 0
    else:
      new_position = children_idx[math.ceil((len(children_idx)) / 2) - 1]

    vertex.x = new_position

  first_layer = sorted(first_layer, key=custom_sort_key_b)

  return (first_layer)


In [16]:
#MAIN ITERATIVE ############################################################
def iterative_crossing_minimization(G, layers,oscm):

  for idx, vertex in enumerate(layers[0]):
    G.update_x_y(vertex.id, idx, 0)

  min_cross = float('inf')
  layers_with_min = None

  all_permutations = permutations(layers[0])
  #for every permutation of the bottom layer
  for perm in all_permutations:
      layers[0] = perm
      pre_layers = None

      #Repeat until the previous ordering is the same as the current ordering
      while pre_layers != layers:
        pre_layers = layers

        #Bottom to top
        for idx, layer in enumerate(layers[:-1]):
          first_layer = layer
          second_layer = layers[idx+1]

          #rearrange the top layer
          if oscm == "Bary":
            second_layer = rearange_of_layers_Barycenter(first_layer, second_layer)
          elif oscm == "Median":
            second_layer = rearange_of_layers_median(first_layer, second_layer)

          layers[idx+1] = second_layer

        #for top to bottom
        for idx in range(len(layers)-1, 0, -1):
          first_layer = layers[idx-1]
          second_layer = layers[idx]

          #rearrange the bottom layer
          if oscm == "Bary":
            first_layer = rearange_of_layers_reverse_Barycenter(first_layer, second_layer )
          elif oscm == "Median":
            first_layer = rearange_of_layers_median_reverse(first_layer, second_layer)

          # Replace the sublist at the found index with the new sublist
          layers[idx-1] = first_layer

      total_cross = calculate_total_crossing(layers)
      #Keep the layers ordering with the minimum crossing
      if total_cross < min_cross:
        min_cross = total_cross
        layers_with_min = layers

  layers = layers_with_min
  print("min", min_cross)


In [17]:
update_parents_children(layered_dummies_graph)
update_parents_children_in_layers(layers, layered_dummies_graph.edges )

#iterative for barycenter
iterative_crossing_minimization(layered_dummies_graph, layers, "Bary")

#iterative median
# iterative_crossing_minimization(layered_dummies_graph, layers, "Median")

min 1844


In [18]:
def visualize_layerd_with_crossing(layers, G, fig_size, radius):

  fig, ax = plt.subplots(figsize=(fig_size, fig_size))  # Adjust width and height as needed
  # Plot Vertices
  scaling_factor = 1 / len(G.vertices)
  for idx,layer in enumerate(layers):
      for idx1,vertex in enumerate(layer):
            #update_x_y
            G.update_x_y(vertex.id, idx1, idx)
            if vertex.id.startswith("D"):
              ax.scatter(idx1, idx, color='red', zorder=2, s=(scaling_factor * 40000 * (fig_size / 15))/1.2,
                        edgecolors='black')
              ax.text(idx1, idx, str(vertex.id), fontsize=(scaling_factor * 40000 * (fig_size / 15)) / 100,
                      ha='center', va='center', zorder=len(G.vertices) * scaling_factor * 2, color='black')
            else:
              ax.scatter(idx1, idx, color='blue', zorder=2, s=scaling_factor * 40000 * (fig_size / 15),
                      edgecolors='black')
              ax.text(idx1, idx, str(vertex.id), fontsize=(scaling_factor * 40000 * (fig_size / 15)) / 50,
                    ha='center', va='center', zorder=len(G.vertices) * scaling_factor * 2, color='white')


  for edge in G.edges:
        start = edge.start
        end = edge.end
        dx = end.x - start.x
        dy = end.y - start.y
        ax.annotate("", xy=(end.x, end.y), xytext=(start.x, start.y),
                    arrowprops=dict(arrowstyle="->", linewidth=1.5, color='gray'))


  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_title('Graph Visualization with Edge Direction')
  ax.grid(True)
  plt.show()



fig_size=5+len(layered_dummies_graph.vertices)/6 # new line
radius=5+len(layered_dummies_graph.vertices)/4 # new line


In [ ]:
visualize_layerd_with_crossing(layers, layered_dummies_graph, fig_size, radius)

In [20]:
def visualize_layerd_with_crossing_without_dummies(ax,layers, G, fig_size, radius):


  # Plot Vertices
  scaling_factor = 1 / len(G.vertices)
  for idx,layer in enumerate(layers):
      for idx1,vertex in enumerate(layer):
            #update_x_y
            G.update_x_y(vertex.id, idx1, idx)
            if vertex.id.startswith("D"):
              continue
            else:
              ax.scatter(idx1, idx, color='blue', zorder=2, s=50,
                      edgecolors='black')
              ax.text(idx1, idx, str(vertex.id), fontsize=(scaling_factor * 40000 * (fig_size / 15)) / 100,
                    ha='center', va='center', zorder=len(G.vertices) * scaling_factor * 2, color='white')


  for edge in G.edges:
      start = edge.start
      end = edge.end
      dx = end.x - start.x
      dy = end.y - start.y
      if end.id.startswith("D"):
        ax.plot([start.x, end.x], [start.y, end.y],color='gray', zorder=1, linewidth=0.5)
      else:
        ax.annotate("", xy=(end.x, end.y), xytext=(start.x, start.y),
                    arrowprops=dict(arrowstyle="->", linewidth=0.5, color='gray'))


  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_title('Graph Visualization with Edge Direction')
  ax.grid(True)
  plt.show()





Step 6


In [21]:
# Parsing the data from the dot file:
G = pydot.graph_from_dot_file(url)[0]
#Convering the graph into our custom graph
graph1 = create_custom_graph(G) #O(V) + O(E) = O(V + E)
# Visualize the graph

fig_size=5+len(graph1.vertices)/6 # new line
radius=5+len(graph1.vertices)/4 # new line

#graph1.visualize_graph(fig_size,radius) #O(V + E)
# Parsing the data from the dot file:
G = pydot.graph_from_dot_file(url)[0]
#Convering the graph into our custom graph
graph3 = create_custom_graph(G) #O(V) + O(E) = O(V + E)
# Visualize the graph

fig_size=5+len(graph3.vertices)/6 # new line
radius=5+len(graph3.vertices)/4 # new line

#graph1.visualize_graph(fig_size,radius) #O(V + E)



In [22]:
def floyd_warshall(G):

  D_array = np.full((len(G.vertices), len(G.vertices)), np.inf)

  for edge in G.edges:
      D_array[int(edge.start.id)-1][int(edge.end.id)-1] = edge.weight

      D_array[int(edge.end.id)-1][int(edge.start.id)-1] = edge.weight

  for v in G.vertices:
    D_array[int(v.id)-1][int(v.id)-1] = 0.0

  for k in G.vertices:
    for i in G.vertices:
      for j in G.vertices:
        k_idx = int(k.id) - 1
        i_idx = int(i.id) - 1
        j_idx = int(j.id) - 1

        if D_array[i_idx][j_idx] > D_array[i_idx][k_idx] + D_array[k_idx][j_idx]:
          D_array[i_idx][j_idx] = D_array[i_idx][k_idx] + D_array[k_idx][j_idx]
         # print("inside Floyd Darr: ",D_array[i_idx][j_idx])

  return(D_array)

for v in graph1.vertices:
  if v.id == '"\\n"':
      graph1.vertices.remove(v)

D_array = floyd_warshall(graph1)

for v in graph3.vertices:
  if v.id == '"\\n"':
      graph3.vertices.remove(v)

D_array3 = floyd_warshall(graph3)

print(D_array)


[[0. 1. 9. ... 9. 8. 8.]
 [1. 0. 8. ... 8. 7. 7.]
 [9. 8. 0. ... 6. 5. 5.]
 ...
 [9. 8. 6. ... 0. 3. 3.]
 [8. 7. 5. ... 3. 0. 2.]
 [8. 7. 5. ... 3. 2. 0.]]


In [23]:
from sklearn.manifold import MDS

def projection_visualization(ax1,X_tranformed, D_array, vertices, edges):


    for i, vertex in enumerate(vertices):
        ax1.scatter(X_transformed[i, 0], X_transformed[i, 1], color='blue')
        ax1.text(X_transformed[i, 0], X_transformed[i, 1], f'{vertex.id}', fontsize=9)

    for edge in edges:
        ax1.plot([edge.start.x, edge.end.x], [edge.start.y, edge.end.y], color='gray', alpha=0.5)


    ax1.set_title('MDS Visualization Les Miserables')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.grid(True)



In [24]:

import numpy as np
from sklearn.manifold import TSNE


# Visualize the embedded points
def visualize_TSNE(ax2,X_transformed1, vertices, edges):


    ax2.scatter(X_transformed1[:, 0], X_transformed1[:, 1], c='b', s=50)
    for i, vertex in enumerate(vertices):
        ax2.text(X_transformed1[i, 0], X_transformed1[i, 1], str(vertex.id))

    for edge in edges:
        ax2.plot([edge.start.x, edge.end.x], [edge.start.y, edge.end.y], color='gray', alpha=0.5)
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_title('TSNE Visualization Les Miserables')
    ax2.grid(True)




In [ ]:
# Plot THE GRAPHS FROM STEP 3, 4 AND 6
fig, (ax1_combined, ax2_combined, ax3_combined, ax4_combined) = plt.subplots(1, 4, figsize=(48, 18))

#
# First sebplot - STEP 3
G = pydot.graph_from_dot_file(url)[0]
graph = create_custom_graph(G)
for v in graph.vertices:
  if v.id == '"\\n"':
      graph.vertices.remove(v)

create_forced_graph(ax1_combined, graph, 1000)

#Second subplot - STEP 6 - MDS


#dissimilarity = 1 - D_array
mds = MDS(n_components=2, dissimilarity='precomputed', max_iter = 1000, eps = 0.0001)
X_transformed = mds.fit_transform(D_array)

for i in range(len(X_transformed)):
  for v in graph1.vertices:
    if v.id == str(i+1):
      v.x = X_transformed[i][0]
      v.y = X_transformed[i][1]

convergences = mds.stress_
print("Convergences:", convergences)
print("Best_iter:", mds.n_iter_)

projection_visualization(ax2_combined,X_transformed, D_array, graph1.vertices, graph1.edges)

#Third subplot - STEP 6 - TSNE
tsne = TSNE(n_components=2,init="random",metric="precomputed", perplexity = len(graph.vertices)/2.2, n_iter = 5000, learning_rate = 50  ,min_grad_norm = 0.0000000001)
X_transformed1 = tsne.fit_transform(D_array3)

for i in range(len(X_transformed1)):
  for v in graph3.vertices:
    if v.id == str(i+1):
      v.x = X_transformed1[i][0]
      v.y = X_transformed1[i][1]


visualize_TSNE(ax3_combined,X_transformed1, graph3.vertices, graph3.edges )

print("KL Divergence:", tsne.kl_divergence_)
print("Number of iterations:", tsne.n_iter_)
print("learning rate", tsne.learning_rate_)

#Forth subplot - step 4
fig_size=5+len(layered_dummies_graph.vertices)/6 # new line
radius=5+len(layered_dummies_graph.vertices)/4 # new line
visualize_layerd_with_crossing_without_dummies(ax4_combined ,layers, layered_dummies_graph, fig_size, radius)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import pearsonr



# GRAPH QUALITY METRICS - pending crossing resolution and stress of layout ===================================================================
# Count crosings########################
def orientation(node1, node2, node3):
    val = (node2.y - node1.y) * (node3.x - node2.x) - (node2.x - node1.x) * (node3.y - node2.y)
    if val == 0:
        return 0
    return 1 if val > 0 else 2

def on_segment(node1, node2, node3):
    if (node2.x <= max(node1.x, node3.x) and node2.x >= min(node1.x, node3.x) and
        node2.y <= max(node1.y, node3.y) and node2.y >= min(node1.y, node3.y)):
        return True
    return False

def intersect(start1, end1, start2, end2):
    if start1 == start2 or start1 == end2 or end1 == start2 or end1 == end2:
        return False

    o1 = orientation(start1, end1, start2)
    o2 = orientation(start1, end1, end2)
    o3 = orientation(start2, end2, start1)
    o4 = orientation(start2, end2, end1)

    if o1 != o2 and o3 != o4:
        return True

    if o1 == 0 and on_segment(start1, start2, end1):
        return True

    if o2 == 0 and on_segment(start1, end2, end1):
        return True

    if o3 == 0 and on_segment(start2, start1, end2):
        return True

    if o4 == 0 and on_segment(start2, end1, end2):
        return True

    return False

def count_crossings(edges):
    count = 0
    n = len(edges)
    for i in range(n - 1):
        for j in range(i + 1, n):
            if intersect(edges[i].start, edges[i].end, edges[j].start, edges[j].end):
                count += 1
    return count


crossings= count_crossings(graph.edges)
print("force directed crossings : ",crossings)
crossings= count_crossings(layered_dummies_graph.edges)
print("Layerd crossings : ",crossings)
crossings= count_crossings(graph1.edges)
print("MDS crossings : ",crossings)
crossings= count_crossings(graph3.edges)
print("t-SNE crossings : ",crossings)


 # Count crosings########################

 # Cross resolution ########################
def vector(p1, p2):

    return np.array([p2.x - p1.x, p2.y - p1.y])

def dot_product(v1, v2):
    return np.dot(v1, v2)

def magnitude(v):
    magnitude = np.linalg.norm(v)
    return magnitude

def angle_between(v1, v2):
    dot = dot_product(v1, v2)
    mag1 = magnitude(v1)
    mag2 = magnitude(v2)
    if mag1 == 0 or mag2 == 0:
        return 0
    cos_angle = dot / (mag1 * mag2)
    cos_angle = max(min(cos_angle, 1), -1)
    # Calculate the angle in radians
    angle_radians = np.arccos(cos_angle)
    # convert rad to degrees
    angle_between_vectors = np.degrees(angle_radians)
    angle_between_vectors = min(angle_between_vectors, 180 - angle_between_vectors)

    return angle_between_vectors
def crossing_resolution(edges):
    min_angle = 180
    for i in range(len(edges) - 1):
        for j in range(i + 1, len(edges)):
          if intersect(edges[i].start,edges[i].end,edges[j].start,edges[j].end):

              v1 = vector(edges[i].start, edges[i].end)
              v2 = vector(edges[j].start, edges[j].end)
              angle = angle_between(v1, v2)
             # print("angle",angle)
              if angle != 0:
                min_angle = min(min_angle, angle)


    return min_angle



Angle_res= crossing_resolution(graph.edges)
print("Angle resolution in force directed graph -- The smallest angle from crossing edges in the graph : ",Angle_res)
Angle_res= crossing_resolution(layered_dummies_graph.edges)
print("Angle resolution in layerd graph -- The smallest angle from crossing edges in the graph : ",Angle_res)
Angle_res= crossing_resolution(graph1.edges)
print("Angle resolution in MDS projection -- The smallest angle from crossing edges in the graph : ",Angle_res)
Angle_res= crossing_resolution(graph3.edges)
print("Angle resolution in t-SNE projection -- The smallest angle from crossing edges in the graph : ",Angle_res)
 # Cross resolution ########################

# Stress of layout ########################
def euclidean_distance(vertex1, vertex2):
    return ((vertex1.x - vertex2.x) ** 2 + (vertex1.y - vertex2.y) ** 2) ** 0.5

#Give to the graph edges the euclidian distance between start-end as edge weight - needed for the layout stress formula because stress of layouts need the actual distances not the theoritical distances
for edge in graph.edges:
    edge.weight = euclidean_distance(edge.start, edge.end)


for edge in graph1.edges:
    edge.weight = euclidean_distance(edge.start, edge.end)

for edge in graph3.edges:
    edge.weight = euclidean_distance(edge.start, edge.end)

for edge in layered_dummies_graph.edges:
    edge.weight = euclidean_distance(edge.start, edge.end)

def calculate_stress_of_layout(graph,D_array):
    stress = 0
   # min_ideal = 100000000
    for i in range(len(D_array)-1):
        for j in range(i + 1,len(D_array[i])):
            if D_array[i][j] != 0:

                v1=graph.get_vertex(str(i+1))
                v2=graph.get_vertex(str(j+1))
                ideal = max(0.01,euclidean_distance(v1,v2)) # in case we have overlap ideal will be 0 so we set to take a value close to zero in that case

                #print("D_array: ",D_array[i][j],"ideal: ",ideal)
                #if ideal < min_ideal: min_ideal=ideal

                stress += (1/ideal**2) * (D_array[i][j]-ideal)**2 # na to tsekarw aurio an einai swsto
   # print("min ideal: ",min_ideal)

    return stress

# Give to dummy nodes of the layerd graph a proper node id eg convert from D1_S1_e12  --> 78
i=1
for v in layered_dummies_graph.vertices:
    if v.id.startswith("D"):
       v.id=str(len(graph.vertices)+i)
       i+=1


# end of convertion



D_array_g = floyd_warshall(graph)
stress = calculate_stress_of_layout(graph,D_array_g)
print("stress of layout force directed graph : ",stress)
D_array_dum = floyd_warshall(layered_dummies_graph)
stress = calculate_stress_of_layout(layered_dummies_graph,D_array_dum)
print("stress of layout layerd graph : ",stress)
D_array_mds = floyd_warshall(graph1)
stress = calculate_stress_of_layout(graph1,D_array_mds)
print("stress of layout MDS : ",stress)
D_array_sne = floyd_warshall(graph3)
stress = calculate_stress_of_layout(graph3,D_array_sne)
print("stress of layout t-SNE : ",stress)






# Stress of layout ########################
#----------------------------------------------------------------------------------------------------------------------------------------

# PROJECTIONS QUALITY METRICS ===================================================================
def normalized_stress(highDim_dstance, lowDim_dstance):

    n1 = np.sum((highDim_dstance - lowDim_dstance) ** 2)
    n2 = np.sum(highDim_dstance ** 2)

    stress = n1 / n2

    return stress
#----------------------------------------------------------------------------------------------------------------------------------------
# continuity #######################
def continuity(D_high, D_low, k=5):
    n = D_high.shape[0]  # Number of nodes
    nn_orig = D_high.argsort()  # sort neighbours in high dimentional space
    nn_proj = D_low.argsort()  # Τsort neighbours in low dimentional space

    knn_orig = nn_orig[:, :k + 1][:, 1:]  # K neighbours of each node in high dimentional space
    knn_proj = nn_proj[:, :k + 1][:, 1:]  # K neighbours of each node in low dimentional space

    sum_i = 0

    for i in range(n):
        V = np.setdiff1d(knn_orig[i], knn_proj[i])

        sum_j = 0
        for j in range(V.shape[0]):
            rank_diff = np.where(nn_proj[i] == V[j])[0] - k
            sum_j += rank_diff

        sum_i += sum_j

    continuity = 1 - (2 / (n * k * (2 * n - 3 * k - 1))) * sum_i

    return float(continuity)
# continuity #######################
#----------------------------------------------------------------------------------------------------------------------------------------
# Trustworthiness #######################
def Trustworthiness(D_high, D_low, k=5):
        n = D_high.shape[0]  # Number of nodes
        nn_orig = D_high.argsort() # sort neighbours in high dimentional space
        nn_proj = D_low.argsort()  # sort neighbours in low dimentional space

        knn_orig = nn_orig[:, :k + 1][:, 1:]  # K neighbours of each node in high dimentional space
        knn_proj = nn_proj[:, :k + 1][:, 1:]  # K neighbours of each node in low dimentional space

        sum_i = 0

        for i in range(n):
            V = np.setdiff1d(knn_proj[i], knn_orig[i])

            sum_j = 0
            for j in range(V.shape[0]):
                rank_diff = np.where(nn_orig[i] == V[j])[0] - k
                sum_j += rank_diff

            sum_i += sum_j

        continuity = 1 - (2 / (n * k * (2 * n - 3 * k - 1))) * sum_i

        return float(continuity)
# Trustworthiness #######################
#----------------------------------------------------------------------------------------------------------------------------------------
# CALCULATE METRICS FOR MDS


# Calculate continuity , trustworthiness , normalised stress and Shepard score for MDS


lowDim_dstance=euclidean_distances(X_transformed)
highDim_dstance=D_array# giati xrisimopoioume pinaka pou periexei apostaseis gia na vroume euclidian apostaseis na to dw avrio mallon lathos
#print("x_transformed",X_transformed)
#print("highDim",lowDim_dstance)

stress_MDS = normalized_stress(highDim_dstance, lowDim_dstance)
continuity_MDS = continuity(highDim_dstance,lowDim_dstance,max(len(graph.vertices)//10,1)) # to set k at least 1 for very small graphs
trust = Trustworthiness(highDim_dstance,lowDim_dstance,max(len(graph.vertices)//10,1)) # to set k at least 1 for very small graphs


shepard_distances = lowDim_dstance

# Plot Shepard diagram for MDS
plt.scatter(D_array.flatten(), shepard_distances.flatten())
plt.xlabel('Original distances')
plt.ylabel('Transformed distances')
plt.title('Shepard Diagram')
plt.show()

# Calculate Shepard Goodness score
shepard_correlation, _ = pearsonr(D_array.flatten(), shepard_distances.flatten())
print("Shepard Goodness score for MDS:", shepard_correlation)
print("Trustworthiness MDS :", trust)
print("Normalized Stress for MDS:", stress_MDS)
print("Continuity for DMS:", continuity_MDS)
#----------------------------------------------------------------------------------------------------------------------------------------
# CALCULATE METRICS FOR t-SNE
# Calculate continuity , trustworthiness , normalised stress and Shepard score for t-SNE

lowDim_dstance=euclidean_distances(X_transformed1)
highDim_dstance=D_array3 # # giati xrisimopoioume pinaka pou periexei apostaseis gia na vroume euclidian apostaseis na to dw avrio mallon lathos
stress_SNE = normalized_stress(highDim_dstance, lowDim_dstance)
continuity_SNE = continuity(highDim_dstance,lowDim_dstance, max(len(graph.vertices)//10,1)) # to set k at least 1 for very small graphs
trust = Trustworthiness(highDim_dstance,lowDim_dstance,max(len(graph.vertices)//10,1)) # to set k at least 1 for very small graphs
shepard_distances = lowDim_dstance

# Plot hepard diagram for the t-SNE
plt.scatter(D_array3.flatten(), shepard_distances.flatten())
plt.xlabel('Original distances')
plt.ylabel('Transformed distances')
plt.title('Shepard Diagram')
plt.show()

# Calculate Shepard Goodness score
shepard_correlation, _ = pearsonr(D_array3.flatten(), shepard_distances.flatten())
print("Shepard Goodness score for t-SNE:", shepard_correlation)
print("Trustworthiness t-SNE:", trust)
print("Normalized Stress for t-SNE:", stress_SNE)
print("Continuity for t-SNE:", continuity_SNE)


